In [1]:
from math import *
import matplotlib.pyplot as plt
from scipy.integrate import ode
import time
import matplotlib.pyplot as plt
import gym
import torch as th
import os
from datetime import datetime
import csv
import numpy as np

from stable_baselines3 import TD3, SAC
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import SubprocVecEnv, VecMonitor
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

from bipedalWalkerPMTG import BipedalWalkerPMTG

import base64
from pathlib import Path

from IPython import display as ipythondisplay

from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

/home/yoggi/biped_research/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Addresses

In [2]:
# Put here the address with your learned agent
log_dir = "/home/yoggi/biped_research/models/PMTG_SAC-bipedalWalkerHardcore-model/best_single/2022-10-13_08-09-02_model"
# address where to save a video with the learned agent
video_dir = "/home/yoggi/biped_research/models/PMTG_SAC-bipedalWalkerHardcore-model/best_single/2022-10-13_08-09-02"

Create environment and load model

In [3]:
eval_env = BipedalWalkerPMTG(
    is_hard=True, 
    action_repeat=1, 
    act_noise=0.0, 
    rew_scale=1.0, 
    learn=False,
    vel_ctrl=False,)

model = SAC.load(log_dir)

Start simulation

If you see any warnings about wrapping a monitor, ignore it. It's not a problem in this situation.

In [4]:
it = 0
obs = eval_env.reset()
r = 0
while it < 35:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = eval_env.step(action)
    eval_env.render()
    it += 1/50
    r += rewards

    if done == True:
        eval_env.reset()
        break
print(f"Reward: {r}")

Reward: 305.9833713046205


Make a video with the agent

In [5]:
# Set up fake display; otherwise rendering will fail
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'


def record_video(env, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (gym environment)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  video_env = DummyVecEnv([lambda: env])
  # Start the video at step=0 and record 500 steps
  video_env = VecVideoRecorder(video_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = video_env.reset()
  for _ in range(video_length):
      action, _ = model.predict(obs)
      obs, _, done, _ = video_env.step(action)
      if done == True:
        break

  # Close the video recorder
  video_env.close()

In [6]:
record_video(eval_env, model, video_length=1500, video_folder=video_dir, prefix="bipedWalker_PMTG_SAC")

_XSERVTransSocketUNIXCreateListener: ...SocketCreateListener() failed
_XSERVTransMakeAllCOTSServerListeners: server already running
(EE) 
Fatal server error:
(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) 


Full agent evaluation (mean reward over 100 episodes)

In [7]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100, deterministic=True, render=False, callback=None,
                              reward_threshold=None)
print(f'Mean reward: {mean_reward} +/- {std_reward:.2f}')

/home/yoggi/biped_research/env/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 303.3886143073267 +/- 10.82


In [8]:
eval_env.close()